In [2]:
import numpy as np
import pandas as pd
import re

In [3]:
#Se define la ruta de la información.
data_propiedades = "../Data/properati.csv"
#Leemos los datos del archivo
data = pd.read_csv(data_propiedades, sep=",", encoding="UTF-8")
#Chequeamos que los datos se hayan importado correctamente
data.head(3)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...


In [4]:
data_caba_mask = data.state_name == 'Capital Federal'

data_caba = data.loc[data_caba_mask, :]

### 16. Analisis del campo **description** para sacar cantidad de ambientes y amenities (si es posible) ###

In [5]:
description_series = data_caba['description']
description_series.sample(10)

33663     AMENITIESPiscina climatizada con borde infinit...
79502     VENTA DEPARTAMENTO 2 AMBIENTES EN ALMAGRODepar...
60106     Excelente 2 ambientes al frente con balcon en ...
9541      VENTA  DOS AMBIENTES EN FLORES Unidad monoambi...
116490    Excelente PH dos ambientes en venta! El mismo ...
120815                    Semipiso de 2 ambientes con ba...
25058     Espectacular Dplex en Complejo Tronador, ex fb...
61218     Venta de Departamento 7 AMBIENTES O MAS en Par...
74288     OPORTUNIDAD !!! Ideal para Vivienda o Inversio...
8128      -Depto. en esquina Copernico con balcón perime...
Name: description, dtype: object

In [6]:
amb1_pattern = "((?P<numero_amb>\d)(\s)?(amb|AMB))"
amb1_pattern_regex = re.compile(amb1_pattern)

In [7]:
amb1_match = description_series.apply(lambda x: x if (x is np.NaN) | (x is None) else\
                                      amb1_pattern_regex.search(x))
mask_amb1_match_notnull = amb1_match.notnull()

data_caba.loc[mask_amb1_match_notnull, 'number_rooms'] = \
amb1_match.loc[mask_amb1_match_notnull].apply(lambda x: int(x.group("numero_amb")))

/Users/Nakin/opt/anaconda3/envs/dhdsblend/lib/python3.7/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/Nakin/opt/anaconda3/envs/dhdsblend/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [8]:
data_caba['number_rooms'].sample(15)

7395      NaN
61280     3.0
88375     NaN
86542     NaN
85455     NaN
120833    4.0
9858      NaN
78803     NaN
40773     NaN
62088     2.0
74779     NaN
88629     2.0
40731     NaN
7707      3.0
77054     2.0
Name: number_rooms, dtype: float64

In [9]:
data_caba['number_rooms'].dtypes
data_caba['number_rooms'].notnull().sum()

14830

In [10]:
dorm1_pattern = "((?P<numero_dorm>\d)(\s)?(dorm|DORM))"
dorm1_pattern_regex = re.compile(dorm1_pattern)

In [11]:
dorm1_match = description_series.apply(lambda x: x if (x is np.NaN) | (x is None) else dorm1_pattern_regex.search(x))

mask_dorm1_match_notnull = dorm1_match.notnull()

data_caba.loc[mask_dorm1_match_notnull, 'number_rooms'] = \
dorm1_match.loc[mask_dorm1_match_notnull].apply(lambda x: int(x.group("numero_dorm"))+1)
#se asume que el numero de ambientes es numero_dorm +1 



In [12]:
data_caba['number_rooms'].sample(15)

79230     NaN
119942    NaN
70656     NaN
61313     2.0
78933     NaN
81519     NaN
72126     NaN
102074    3.0
51662     NaN
96912     NaN
15186     3.0
86138     2.0
56227     3.0
44623     NaN
63349     NaN
Name: number_rooms, dtype: float64

In [13]:
data_caba['number_rooms'].notnull().sum()

17337

In [ ]:
#Falta Ambiente unico, mono(\s)?ambiente, 1 y 1/2 amb / dos/tres/cuatro/cinco ([a-zA-Z]*)? dormi

#PISOS: \d piso, planta baja, PH = planta baja, \der\spiso, con palabras?

#Amemities (pileta/piscina, SUM, laundry/lavadero, cochera/garage, Gimnasio/gym, seguridad)

In [14]:
amb2_pattern = "(?P<mono>(mono|MONO)(\s)?(amb|AMB))"
amb2_pattern_regex = re.compile(amb2_pattern)

In [15]:
amb2_match = description_series.apply(lambda x: x if (x is np.NaN) | (x is None) else\
                                      amb2_pattern_regex.search(x))
mask_amb2_match_notnull = amb2_match.notnull()

mask_amb2_match_notnull.sum()


2516

In [16]:
mono_amb = 1
data_caba.loc[mask_amb2_match_notnull, 'number_rooms'] = mono_amb


In [17]:
data_caba['number_rooms'].sample(15)

71958     3.0
36193     NaN
55973     1.0
121026    NaN
48406     3.0
79079     NaN
23484     NaN
60827     NaN
46559     NaN
63751     3.0
109534    1.0
85112     4.0
1776      NaN
62880     2.0
57260     3.0
Name: number_rooms, dtype: float64

In [18]:
data_caba['number_rooms'].notnull().sum()

19287

### Amenities ###

#### Pileta ####

In [19]:
pileta_pattern = "(?P<pileta>pileta|piscina|picina)"
pileta_pattern_regex = re.compile(pileta_pattern)

In [20]:
pileta_match = description_series.apply(lambda x: x if (x is np.NaN) | (x is None) else\
                                      pileta_pattern_regex.search(x))
mask_pileta_match_notnull = pileta_match.notnull()

data_caba.loc[mask_pileta_match_notnull, 'pileta'] = \
pileta_match.loc[mask_pileta_match_notnull].apply(lambda x: x.group("pileta"))

In [21]:
data_caba['pileta'].sample(15)

97813     piscina
64740         NaN
11846         NaN
103499    piscina
78315     piscina
73950     piscina
117527        NaN
89345         NaN
87041         NaN
33923         NaN
53154         NaN
23438         NaN
1274          NaN
96339         NaN
32717         NaN
Name: pileta, dtype: object

In [22]:
data_caba['pileta'].notnull().sum()

4004

#### Laundry ####

In [23]:
laudry_pattern = "(?P<laundry>laundry|lavadero)"
laundry_pattern_regex = re.compile(laudry_pattern)

In [24]:
laundry_match = description_series.apply(lambda x: x if (x is np.NaN) | (x is None) else\
                                      laundry_pattern_regex.search(x))
mask_laundry_match_notnull = laundry_match.notnull()

In [25]:
mask_laundry_match_notnull.sum()

6174

In [26]:
data_caba.loc[mask_laundry_match_notnull, 'laundry'] = \
laundry_match.loc[mask_laundry_match_notnull].apply(lambda x: x.group("laundry"))

In [27]:
data_caba['laundry'].sample(15)

36978         NaN
34491     laundry
83919         NaN
75993         NaN
48877         NaN
58503         NaN
71425         NaN
42378     laundry
119422        NaN
23274         NaN
80930         NaN
60019     laundry
120762        NaN
48844         NaN
79866         NaN
Name: laundry, dtype: object

In [28]:
data_caba['laundry'].notnull().sum()

6174

#### Parking ####

In [29]:
parking_pattern = "(?P<parking>parking|estacionamiento|garage|cochera)"
parking_pattern_regex = re.compile(parking_pattern)

parking_match = description_series.apply(lambda x: x if (x is np.NaN) | (x is None) else\
                                      parking_pattern_regex.search(x))
mask_parking_match_notnull = parking_match.notnull()
mask_parking_match_notnull.sum()

5797

In [30]:
data_caba.loc[mask_parking_match_notnull, 'parking'] = \
parking_match.loc[mask_parking_match_notnull].apply(lambda x: x.group("parking"))

data_caba['parking'].sample(15)

78140         NaN
5815          NaN
58795         NaN
49873     cochera
353           NaN
97380         NaN
86211         NaN
102534        NaN
19543         NaN
26402         NaN
85330         NaN
118092        NaN
94961         NaN
19694         NaN
112341    cochera
Name: parking, dtype: object

In [31]:
data_caba['parking'].notnull().sum()

5797

#### Gimnasio ####

In [32]:
gym_pattern = "(?P<gimnasio>gim|gym|gimnasio|fitness)"
gym_pattern_regex = re.compile(gym_pattern)

gym_match = description_series.apply(lambda x: x if (x is np.NaN) | (x is None) else\
                                      gym_pattern_regex.search(x))
mask_gim_match_notnull = gym_match.notnull()
mask_gim_match_notnull.sum()

1690

In [33]:
data_caba.loc[mask_gim_match_notnull, 'gimnasio'] = \
gym_match.loc[mask_gim_match_notnull].apply(lambda x: x.group("gimnasio"))

data_caba['gimnasio'].sample(15)

59404     NaN
47670     gim
92118     gim
8340      NaN
53581     gym
27102     NaN
86717     NaN
47342     NaN
18062     NaN
116430    NaN
41196     NaN
70946     NaN
84179     NaN
27202     NaN
97046     NaN
Name: gimnasio, dtype: object

In [34]:
data_caba['gimnasio'].notnull().sum()

1690